In [2]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px

In [3]:
genres_data = pd.read_csv("./dataset/top_genres_in_the_world.csv")


genres_columns = ['Hip hop/Rap/R&b', 'EDM', 'Pop', 'Rock/Metal', 'Latin/Reggaeton', 'Other']
genres_data_subset = genres_data[genres_columns]
row_sums = genres_data_subset.sum(axis=1)
normalized_genres_data = genres_data_subset.div(row_sums, axis=0).mul(100, axis=0)
normalized_genres_data['Country'] = genres_data['Country']
countries_and_pop = normalized_genres_data[['Country', 'Pop']]

# Create the world map plot using Plotly
fig1_pop = px.choropleth(countries_and_pop, 
                    locations="Country", 
                    locationmode="country names", 
                    color="Pop", 
                    range_color=[0, 100],
                    title="Popularity of Pop Music Genres in the World"
                   )
fig1_pop.update_layout(
    # height=400,
    # width=600,
    plot_bgcolor='#333', paper_bgcolor='#333',
    title_font=dict(size=25, color='#8a8d93', family="Lato, sans-serif"),
    font=dict(color='#8a8d93'),
    height=600,
    
    margin=dict(l=15, r=15, t=50, b=10, pad=0)
)

fig1_pop.show()

In [4]:
countries_and_edm = normalized_genres_data[['Country', 'EDM']]

# Create the world map plot using Plotly
fig2_edm = px.choropleth(countries_and_edm, 
                    locations="Country", 
                    locationmode="country names", 
                    color="EDM", 
                    range_color=[0, 100],
                    title="Popularity of EDM Music Genres in the World"
                   )
fig2_edm.update_layout(
    # height=400,
    # width=600,
    plot_bgcolor='#333', paper_bgcolor='#333',
    title_font=dict(size=25, color='#8a8d93', family="Lato, sans-serif"),
    font=dict(color='#8a8d93'),
    height=600,
    
    margin=dict(l=15, r=15, t=50, b=10, pad=0)
)

fig2_edm.show()

In [10]:
countries_and_rock = normalized_genres_data[['Country', 'Rock/Metal']]

# Create the world map plot using Plotly
fig3_rock = px.choropleth(countries_and_rock, 
                    locations="Country", 
                    locationmode="country names", 
                    color='Rock/Metal', 
                    range_color=[0, 100],
                    title="Popularity of Rock/Metal Music Genres in the World"
                   )
fig3_rock.update_layout(
    # height=400,
    # width=600,
    plot_bgcolor='#333', paper_bgcolor='#333',
    title_font=dict(size=25, color='#8a8d93', family="Lato, sans-serif"),
    font=dict(color='#8a8d93'),
    height=600,
    
    margin=dict(l=15, r=15, t=50, b=10, pad=0)
)

fig3_rock.show()

In [11]:
countries_and_hiphop = normalized_genres_data[['Country', 'Hip hop/Rap/R&b']]

# Create the world map plot using Plotly
fig4_hiphop = px.choropleth(countries_and_hiphop, 
                    locations="Country", 
                    locationmode="country names", 
                    color='Hip hop/Rap/R&b', 
                    range_color=[0, 100],
                    title="Popularity of 'Hip hop/Rap/R&B' Music Genres in the World"
                   )
fig4_hiphop.update_layout(
    # height=400,
    # width=600,
    plot_bgcolor='#333', paper_bgcolor='#333',
    title_font=dict(size=25, color='#8a8d93', family="Lato, sans-serif"),
    font=dict(color='#8a8d93'),
    height=600,
    
    margin=dict(l=15, r=15, t=50, b=10, pad=0)
)

fig4_hiphop.show()

In [12]:
path_dataset2 = "./dataset/albumlist.csv"
dataset = pd.read_csv(path_dataset2)

In [13]:
genres_list = dataset['Genre'].value_counts()
count_map = {}
for genres in genres_list.keys():
    count = genres_list[genres]
    genres = genres.strip()
    split_genres = genres.split(',')
    for genre in split_genres:
        genre = genre.strip(" & ")
        if genre in count_map.keys():
            count_map[genre] = count_map[genre] + count
        else:
            count_map[genre] = count
count_map = dict(sorted(count_map.items(), key=lambda item: item[1], reverse=True))

In [14]:
import plotly.express as px

# Create a new DataFrame to store the data for the plot
data = []

# Specify the order of genres
all_genres = ["Rock","Funk / Soul","Pop","Electronic"]

# Loop over each decade
for start_year in range(1960, 2001, 10):
    end_year = start_year + 9
    filtered_dataset = dataset[(dataset['Year'] >= start_year) & (dataset['Year'] <= end_year)]
    genre_albums = {genre: 0 for genre in all_genres}
    counts = filtered_dataset['Genre'].value_counts()
    for genres in counts.index:
        count = counts[genres]
        genres = genres.strip()
        split_genres = genres.split(',')
        for genre in split_genres:
            genre = genre.strip(" & ")
            if genre in genre_albums:
                genre_albums[genre] += count
    for genre, count in genre_albums.items():
        data.append({'Decade': str(start_year) + 's', 'Genre': genre, 'Number of Albums': count})

# Convert the data to a DataFrame
df = pd.DataFrame(data)
df = df[df['Genre'].isin(all_genres)]
# Create the animated bar chart
fig5_genre_list = px.bar(df, x='Number of Albums', y='Genre', animation_frame='Decade', orientation='h', 
             range_x=[0, df['Number of Albums'].max()], 
             category_orders={'Genre': all_genres})

# Show the plot
fig5_genre_list.show()

In [15]:
#application integration
import dash
from dash import Dash, dcc, html, Input, Output
import base64


In [16]:

external_stylesheets = [
    'https://codepen.io/chriddyp/pen/bWLwgP.css',
    {
        'href': 'https://stackpath.bootstrapcdn.com/bootstrap/4.1.3/css/bootstrap.min.css',
        'rel': 'stylesheet',
        'integrity': 'sha384-MCw98/SFnGE8fJT3GXwEOngsV7Zt27NXFoaoApmYm81iuXoPkFOJwJ8ERdknLPMO',
        'crossorigin': 'anonymous'
    }
]

with open("./dataset/music_first image.jpg", "rb") as image_file:
    encoded_image = base64.b64encode(image_file.read()).decode()

In [25]:
app = Dash(__name__,external_stylesheets =external_stylesheets)
styles = {
    'dropdown': {
        'width': '420px',
       ' padding-left':'20px',
        'display': 'inline-block',
        'font-size': '15px','border-color': '#2A547E','border-width': '2px',
        
    },
    'container': {
        'display': 'flex',
        'width': '420px',
        ' margin-left':'20px',
    }
}
app.layout = html.Div([
    html.Div(
        [   # Earth's image in the first line
    html.Img(src=f"data:image/jpg;base64,{encoded_image}", style={"height": "300px", "display": "block", "margin": "auto"}, ),
            
            html.H1("MUSIC EVOLUTION ANALYTICS", style={"text-align": "center","font-family":"PT Sans Narrow",'font-size': '45px' }),
            # Track changes in Earth's temperature, carbon emissions & Sea Levels over time in the third line
            html.H4("Analysis of Evolution of Commercial Music in the World", style={"text-align": "center","font-family":"PT Sans Narrow",'font-size': '20px'}),
        ],
        style={"padding-top": "10px", 'padding-bottom': '10px',"background-color": "black", "color": "white", 'box-shadow': '5px 5px 5px grey',"border-radius": "15px",}
    ),
    dcc.Dropdown(
            id="demo-dropdown",
            options=[
                {"label": "Global Music General Info", "value": "general_info"},
                {"label": "Music: Past vs Present", "value": "past_present"},
                {"label": "Case Studies", "value": "case_study"},
                {"label": "Spotify", "value": "spotify"}
            ],
            value="",
            placeholder="Select the Desired Visulization",
        style={'width':'450px',"margin-top": "20px", 'margin-bottom': '20px','padding-left': '20px','font-size': '15px','border-color': '#2A547E','border-width': '2px' }
        ),
    html.Div(id='dd-output-container'),
    
],        style={"background-color": "#CDDEEE", "padding": "10px"}

)


@app.callback(
    Output('dd-output-container', 'children'),
    Input('demo-dropdown', 'value')
)
def update_output(value):
    if value == 'general_info':
        return html.Div(

            children=[
                html.Div(
                            children=[
                                dcc.Graph(id='=pop',figure=fig1_pop,style={"margin-bottom": "10px",'border': '3px solid #2A547E',}),
                                dcc.Graph(id='edm',figure=fig2_edm,style={"margin-bottom": "10px",'border': '3px solid #2A547E'}),
                                
                            ],
                            style={"display": "flex", "flex-direction": "row", "justify-content": "space-between", "width": "100%",},
                        ),
                html.Div(
                            children=[
                                dcc.Graph(id='=rock',figure=fig3_rock,style={"margin-bottom": "10px",'border': '3px solid #2A547E',}),
                                dcc.Graph(id='hiphop',figure=fig4_hiphop,style={"margin-bottom": "10px",'border': '3px solid #2A547E'}),
                            ],
                            style={"display": "flex", "flex-direction": "row", "justify-content": "space-between", "width": "100%",},
                        ),
                html.H1(children='Top Albums of Different Generes over the years',
                            style={'font-size': '20px', 'color': 'white','padding-left':'10px'}),
                dcc.Graph(id='top_albums_genres', figure=fig5_genre_list,style={"margin-bottom": "10px",'border': '3px solid #2A547E'}),


     
        ],style={'background-color': '#4482C1'})
    


    elif value == 'past_present':
        return html.Div(
            children=[
                
            ],style={'background-color': '#4482C1'}          
        )
    elif value == 'case_study':
        return html.Div(
            children=[
            ],
            style={'background-color': '#4482C1'},

        )
    elif value == 'spotify':
        return html.Div(
            children=[

            ],style={'background-color': '#4482C1'})
    else:
        return html.Div()


In [26]:
if __name__ == '__main__':
    app.run_server()